In [1]:
from json import load
import queue
from database import *
import sklearn.preprocessing
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from queue import Queue
import loader

available = False
last_centroids = None
centroids_queue = Queue(100)

query = f"""
    SELECT block_hash, wallets.*
    FROM 
        (SELECT DISTINCT transactions.block_hash, inputs.address
        FROM inputs, transactions
        WHERE 
        inputs.transaction_id == transactions.id
        UNION 
        SELECT DISTINCT transactions.block_hash, outputs.address
        FROM outputs, transactions
        WHERE 
        outputs.transaction_id == transactions.id
        ) AS io, wallets
    WHERE 
    io.address == wallets.addr
    ORDER BY block_hash, wallets.addr
"""

wallets_df = pd.read_sql_query(query, engine)
wallets_data = wallets_df.drop(['addr', 'block_hash'], axis=1).to_numpy()
X = sklearn.preprocessing.normalize(wallets_data)

In [3]:
centroids, labels, inertia = (None, None, None)
for n in range(100):
    print(n)

    centroids, labels, inertia = sklearn.cluster.k_means(X, 4, init=last_centroids if last_centroids is not None else 'random', 
                                                                        max_iter=1,
                                                                        n_init=1)
                                                                        
    

    if n < 2  or np.linalg.norm(centroids-last_centroids) > 1e-6 :
        last_centroids = centroids
        centroids_queue.put((centroids, False))
    else:
        centroids_queue.put((centroids, True))
        available = True
        break


0
1
2
3
4
5
6
7
8
9
10


In [5]:
centroids, last = (last_centroids, True)




X = sklearn.preprocessing.normalize(wallets_data)
_, labels, _ = sklearn.cluster.k_means(X, len(centroids), init=centroids, max_iter=1, n_init=1)

wallets_df = wallets_df.assign(cluster=labels.astype(int)   )
wallets_df.groupby('cluster').mean()


,avg_vin,avg_vout,var_vin,var_vout,unique_deg_in,unique_deg_out,deg_in,deg_out,balance,received_value,total_txs
cluster,,,,,,,,,,,
0,0.099410,0.103716,0.077922,0.114968,52.657667,4.729099,60.097775,7.582735,0.050718,1.195847,11.156411
1,0.024488,0.026420,0.015175,0.015526,6.969446,5.902463,9.654408,9.349994,0.013031,0.322099,8.644686
2,0.233347,0.233300,0.168922,0.205919,3.673833,23.276197,4.496514,24.340353,0.013789,1.601175,5.378381
3,8.758205,8.896544,14609.897543,14388.843824,30.470974,14.968400,42.288294,25.697104,31.728710,194169.345259,4201.801411
